In [2]:
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import requests
payload = {
    "priceDate.month": 6,
    "priceDate.day": 17,
    "priceDate.year": 2025,
    "submit": "submit"
}

url = "https://www.treasurydirect.gov/GA-FI/FedInvest/selectSecurityPriceDate"

try:
    response = requests.post(url=url, data=payload, timeout=30)
    response.raise_for_status()

except requests.exceptions.RequestException as e:
    print(f"Error occured: {e}")

In [2]:
output = BeautifulSoup(response.text, "html.parser")
table = output.find("table", class_="data1") # Getting table from webpage

header_row = table.find_all("th")
headers = [th.get_text(strip=True) for th in header_row]

In [9]:
data_rows = table.find_all("tr")[1:]
all_rows = list()
for tr in data_rows:
    row_data = [td.get_text(strip=True) for td in tr.find_all("td")]
    all_rows.append(row_data)

df = pd.DataFrame(data=all_rows, columns=headers)

In [16]:
from datetime import timedelta

datetime.now().date() - timedelta(days=2)


datetime.date(2025, 6, 17)